In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [3]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads
books = db.books

In [4]:
books.count()

10000

In [5]:
books.find_one()

{'_id': ObjectId('5b88855b21c3f05eeaf945eb'),
 'book_id': 1,
 'goodreads_book_id': 2767052,
 'best_book_id': 2767052,
 'work_id': 2792775,
 'books_count': 272,
 'isbn': 439023483,
 'isbn13': 9780439023480.0,
 'authors': 'Suzanne Collins',
 'original_publication_year': 2008.0,
 'original_title': 'The Hunger Games',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'language_code': 'eng',
 'average_rating': 4.34,
 'ratings_count': 4780653,
 'work_ratings_count': 4942365,
 'work_text_reviews_count': 155254,
 'ratings_1': 66715,
 'ratings_2': 127936,
 'ratings_3': 560092,
 'ratings_4': 1481305,
 'ratings_5': 2706317,
 'image_url': 'https://images.gr-assets.com/books/1447303603m/2767052.jpg',
 'small_image_url': 'https://images.gr-assets.com/books/1447303603s/2767052.jpg'}

In [323]:
ratings = db.ratings
booktags = db.booktags
tags = db.tags

In [328]:
#pip install mongojoin


The following command must be run outside of the IPython shell:

    $ pip install mongojoin, MongoCollection

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more information on how to install packages:

    https://docs.python.org/3/installing/


In [339]:
import pprint

In [352]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 8399, 'Title': 'First Among Equals'}
{'Book ID': 6640, 'Title': 'Lady Friday'}
{'Book ID': 4115, 'Title': 'The Sixth Extinction: An Unnatural History'}
{'Book ID': 7638, 'Title': 'The Algebraist'}
{'Book ID': 4606, 'Title': 'Paula'}


In [353]:
for book in db.books.aggregate([{'$sample':{'size':5}},
                    {'$lookup': {
                        'from': 'ratings',
                        'localField': 'book_id',
                        'foreignField': 'book_id',
                        'as': 'joined'},
                    },
                    {'$project':{'_id':0,'Book ID':'$book_id','Title':'$original_title'}}
                   ]):
        pprint.pprint(book)

{'Book ID': 7176, 'Title': 'The Monk'}
{'Book ID': 3896, 'Title': 'Living History'}
{'Book ID': 5076, 'Title': 'Brief Interviews with Hideous Men'}
{'Book ID': 3841, 'Title': 'A Prisoner of Birth'}
{'Book ID': 2148, 'Title': 'Monster'}


In [406]:
for book in db.books.find({"book_id":2}):
    print('Book ID: ', book['book_id'])
    print('Title: ', book['original_title'])

Book ID:  2
Title:  Harry Potter and the Philosopher's Stone


In [437]:
def find_book_info(ID):
    for book in db.books.find({'book_id':ID}):
        print (book)

In [438]:
find_book_info(2)

{'_id': ObjectId('5b88855b21c3f05eeaf945ec'), 'book_id': 2, 'goodreads_book_id': 3, 'best_book_id': 3, 'work_id': 4640799, 'books_count': 491, 'isbn': 439554934, 'isbn13': 9780439554930.0, 'authors': 'J.K. Rowling, Mary GrandPré', 'original_publication_year': 1997.0, 'original_title': "Harry Potter and the Philosopher's Stone", 'title': "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", 'language_code': 'eng', 'average_rating': 4.44, 'ratings_count': 4602479, 'work_ratings_count': 4800065, 'work_text_reviews_count': 75867, 'ratings_1': 75504, 'ratings_2': 101676, 'ratings_3': 455024, 'ratings_4': 1156318, 'ratings_5': 3011543, 'image_url': 'https://images.gr-assets.com/books/1474154022m/3.jpg', 'small_image_url': 'https://images.gr-assets.com/books/1474154022s/3.jpg'}


In [441]:
def find_book_rating(ID):
    for book in db.books.find({'book_id':ID}):
        print (book['average_rating'])

In [411]:
def find_book_title(ID):
    for book in db.books.find({'book_id':ID}):
#        print('Book ID: ', book['book_id'])
        print(book['original_title'])

In [412]:
find_book_title(2)

Harry Potter and the Philosopher's Stone


In [32]:
ratings.find_one()['book_id'],ratings.find_one()['rating']

(258, 5)

In [198]:
#rating_cursor = ratings.find()
rating_cursor = ratings.aggregate([{'$sample':{'size':5}}])                       

In [199]:
from collections import defaultdict
rating_dict = defaultdict(list)
#rating_dict = {}

for rating in rating_cursor:
    print(rating['user_id'],rating['book_id'], rating['rating'])
 #   if rating['book_id'] not in rating_dict:
  #      rating_dict[str(rating['book_id'])] = rating['rating']
  #  else:
  #      rating_dict[str(rating['book_id'])].append(rating['rating'])
    
#    print (type(rating_dict[rating['book_id']]))

24566 358 5
5616 4159 3
17753 90 1
20522 5934 5
28824 86 3


In [170]:
for k, v in rating_dict.items():
    print ('Keys: ', k, 'Ratings: ', v)

Keys:  42 Ratings:  4
Keys:  912 Ratings:  5
Keys:  221 Ratings:  5
Keys:  7723 Ratings:  4
Keys:  402 Ratings:  4


In [161]:
from scipy.sparse import coo_matrix

In [159]:
from scipy.sparse.linalg import svds

In [163]:
import numpy as np

In [495]:
rating_cursor = ratings.find()
#rating_cursor = ratings.aggregate([{'$sample':{'size':5}}])   

In [ ]:
user_list = np.array([])
book_list = np.array([])
rating_list = np.array([])

for rating in rating_cursor:
    user_list = np.append(user_list,(rating['user_id']))
    book_list = np.append(book_list,(float(rating['book_id'])))
    rating_list = np.append(rating_list,(float(rating['rating'])))

In [472]:
user_list

array([16415., 26362., 37198., 17777., 53252.])

In [473]:
rating_list

array([4., 4., 5., 2., 4.])

In [474]:
user_list

array([16415., 26362., 37198., 17777., 53252.])

In [475]:
book_list

array([ 272.,   73., 2478., 1600., 6907.])

In [476]:
user_list.max()

53252.0

In [477]:
mat = coo_matrix((rating_list, (user_list-1, book_list-1)), shape=(int(user_list.max()), int(max(book_list))))

In [478]:
mat.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 4.]])

In [479]:
mat.shape

(53252, 6907)

In [480]:
U, s, VT = svds(mat,k=100)

In [481]:
VT.shape

(100, 6907)

In [482]:
U.shape

(53252, 100)

In [483]:
s.shape

(100,)

In [484]:
s

array([2., 4., 4., 4., 5., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [485]:
VT.shape[1]

6907

Find Similar Items

In [486]:
def find_similar_books(itemID, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        if item != itemID:
            recs.append([item+1,np.dot(VT.T[itemID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [487]:
book_num = 1
num_recom = 20
for book in find_similar_books(book_num,VT,num_recom):
    print(book)
#    print(movies[movies.movieId == str(item[0])].Title,'\n')

(1, 0.016035944827905747)
(6658, 0.005616350295360898)
(37, 0.005546637268570494)
(1147, 0.005446302001750676)
(852, 0.005281443069964975)
(6310, 0.004863842026131715)
(1329, 0.004862747290287774)
(1298, 0.004802999138335555)
(2967, 0.004754692149351942)
(6043, 0.004610055241736656)
(5936, 0.004582862015159991)
(1212, 0.004535096812213384)
(2173, 0.004494139138826673)
(1328, 0.004458432004053792)
(4237, 0.0044544749627672695)
(1761, 0.004436051408114306)
(388, 0.0043717544657749805)
(2743, 0.004342899153924018)
(4131, 0.004278793919749392)
(3230, 0.004264403297756552)


In [488]:
def item_similarity(item1,item2):
    return np.dot(VT.T[item1],VT.T[item2])

In [489]:
item_similarity(2,18)

9.775309546633202e-05

Get Recommendations for Users

In [490]:
def get_recommends_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

In [491]:
USERID=1
num_recom = 20
for item in get_recommends_for_user(USERID,U,VT,num_recom):
    print(item)
    print("Actual Rating: ", mat.toarray()[USERID][item[0]])
    #print(movies[movies.movieId == str(item[0])].Title,'\n')

(2055, 0.001850468730998939)
Actual Rating:  0.0
(1973, 0.001805551770540473)
Actual Rating:  0.0
(4523, 0.0017806898322986027)
Actual Rating:  0.0
(4352, 0.0017782810948929525)
Actual Rating:  0.0
(6548, 0.0017764858483967235)
Actual Rating:  0.0
(2163, 0.0017178366611682412)
Actual Rating:  0.0
(2017, 0.0016368867893164919)
Actual Rating:  0.0
(5215, 0.001626888260165753)
Actual Rating:  0.0
(1635, 0.0015849165675062631)
Actual Rating:  0.0
(135, 0.0015389047105925536)
Actual Rating:  0.0
(423, 0.0015297085761235307)
Actual Rating:  0.0
(3804, 0.0015134545490765009)
Actual Rating:  0.0
(5417, 0.0015090491912795718)
Actual Rating:  0.0
(1626, 0.0015040146748406208)
Actual Rating:  0.0
(4683, 0.0015001699616872008)
Actual Rating:  0.0
(922, 0.0014973840446697183)
Actual Rating:  0.0
(3038, 0.001483801029433646)
Actual Rating:  0.0
(271, 0.0014758026435121947)
Actual Rating:  0.0
(1352, 0.0014599664027197912)
Actual Rating:  0.0
(1852, 0.0014558787346929355)
Actual Rating:  0.0


In [492]:
USERID=73
num_recom = 20
for item in get_recommends_for_user(USERID,U,VT,num_recom):
    #print(item[0])
    find_book_title(item[0])
    find_book_rating(item[0])

Scary Stories to Tell in the Dark
4.05
Corduroy
4.28
Magyk
3.8
ふしぎ遊戯 1
4.08
The Mating
4.02
Manuscrito encontrado em Accra
3.8
Beyond the Highland Mist
3.99
Pawn of Prophecy
4.16
Wings
3.75
The Impossible Knife of Memory
3.93
Under the Banner of Heaven: A Story of Violent Faith
3.98
Mrs. Frisby and the Rats of NIMH
4.13
Du contrat social: ou Principes du droit politique
3.75
Club Dead
4.03
ビューティーポップ 1 [Byūtī poppu 1]
4.15
An Illustrated Life: Drawing Inspiration From The Private Sketchbooks Of Artists, Illustrators And Designers
3.72
NARUTO -ナルト- 巻ノ四十三
4.57
To Tame a Highland Warrior
4.14
Chosen Prey
4.23
Bring Up the Bodies
4.26


In [493]:
find_book_title(1681)

Lady Chatterley's Lover


In [494]:
find_book_rating(1681)

3.49
